In [ ]:
import functions as fun
import datetime
import plotly.express as px

import norm as nm
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import h5py
import tables

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

from sklearn.linear_model import LinearRegression

Cp = 4180

import matplotlib.pyplot as plt

import os

In [ ]:
# Charge et preproc le fichier

file_name = "S425-2ab.csv"
df = fun.pre_proc(file_name)

# or 
# file_path = r"C:\Users\valen\Dropbox (DualSun)\PC\Documents\PVT-PAC\vbusnet-1674040305-semicolon-crlf.xlsx"

df = fun.clean_date_time(df,'Date / Time')
df = fun.drop(df)

# dict_corr.keys()

dict_rename,order,dict_unit = fun.dict_preproc_btp()

df = fun.rename_and_sort(df,dict_rename,order)

In [ ]:
# Sélection de la plage de temps souhaitée

df_a = fun.select(df,datetime.datetime(2022, 11, 24, 15, 50, 00),datetime.datetime(2022, 11, 24, 17, 10, 00))

# df_b = fun.select(df,datetime.datetime(2022, 11, 24, 17, 55, 00),datetime.datetime(2022, 11, 24, 20, 00, 00))

In [ ]:
# Sélection de données vérifiant des conditions

df_a = df.loc[df_a['Flow_rate']>70]
df_a = df.loc[df_a['G']>600]

In [ ]:
# Crée un fichier de données statistiques

Tt = pd.DataFrame()
Tt["T_m"] = (df["T_fluid_in"]+df["T_fluid_out"])/2
Tt["T_m -T_a"] = Tt["T_m"] - df["T_amb"]
Tt["T_m_VFD"] = (df["T_fluid_in"]+df["T_fluid_out_VFD"])/2
Tt["T_m_VFD -T_a"] = Tt["T_m_VFD"] - df["T_amb"]

In [ ]:
# Plot les nuages statistiques des conditions météo

plt.scatter(Tt["T_m -T_a"],df["G"],s=1.)
plt.xlim(-5,15)
plt.show()

plt.scatter(Tt["T_m -T_a"],df["u"],s=1.)
plt.xlim(-5,15)
plt.show()

In [ ]:
# Plot time series data

fig1 = fun.plot_system(df_a,tank=False)
fig2 = fun.plot_meteo(df_a)
fig1.show()
fig2.show()

In [ ]:
# Find steady states

time_list,quasi_list,ss_list = fun.find_steady_states(df)

In [ ]:
df["DT_fluid_out"] = abs(df["T_fluid_out_VFD"] - df["T_fluid_out"])

In [ ]:
hist1 = df['DT_fluid_out'].value_counts(bins=l)

In [ ]:
df.plot.hist(column = "DT_fluid_out",bins=l)

In [ ]:
plt.hist(np.array(df['DT_fluid_out']), bins = l, rwidth = 0.5, density =False)
plt.xlabel('| T_fluid_out - T_fluid_out_VFD |')
plt.show()

In [ ]:
print(df["DT_fluid_out"].mean())
print(df["DT_fluid_out"].std())
print(df["DT_fluid_out"].min())
print(df["DT_fluid_out"].max())
print(df["DT_fluid_out"].idxmax())

In [ ]:
# Save HDF5

address = "S425-2b.h5"
fun.save_h5(address,df_a,dict_unit)